In [1]:
import json
from dataclasses import dataclass, field

In [2]:
with open("event/event_dataset_dev.json") as f:
    data_json = json.load(f)
data_json[0:3]

[{'tid': 2771,
  'info': "If one or more of Ecolab's customers were to experience a disastrous outcome, the firm's reputation could suffer and it could lose multiple customers as a result.",
  'extraInfo': None,
  'labelData': [{'type': 'cause',
    'reason': [[3, 76]],
    'result': [[78, 149]]}]},
 {'tid': 3940,
  'info': "As grocery customers regularly visit the store, they are continually exposed to the firm's higher margin offerings, spurring lucrative general merchandise sales.",
  'extraInfo': None,
  'labelData': [{'type': 'cause',
    'reason': [[49, 114]],
    'result': [[116, 160]]}]},
 {'tid': 710,
  'info': 'Nevertheless, with voices amplified through structural shifts like the rise of digital media, consumers have more agency than ever: if they want LaCroix (or any other National Beverage brand), retailers eventually have to oblige.',
  'extraInfo': None,
  'labelData': [{'type': 'cause',
    'reason': [[18, 61]],
    'result': [[93, 120]]}]}]

In [3]:
ex = data_json[0]
ex

{'tid': 2771,
 'info': "If one or more of Ecolab's customers were to experience a disastrous outcome, the firm's reputation could suffer and it could lose multiple customers as a result.",
 'extraInfo': None,
 'labelData': [{'type': 'cause', 'reason': [[3, 76]], 'result': [[78, 149]]}]}

In [4]:
text = ex["info"]
text

"If one or more of Ecolab's customers were to experience a disastrous outcome, the firm's reputation could suffer and it could lose multiple customers as a result."

In [5]:
@dataclass
class Relation:
    type: str
    causes: list[str]
    effects: list[str]


@dataclass
class Example:
    id: int
    text: str
    relations: list[Relation]


processed = []

for example in data_json:
    id = example["tid"]
    text = example["info"]
    relations = []

    for relation in example["labelData"]:
        type = relation["type"]
        causes = [text[start:end] for start, end in relation["reason"]]
        effects = [text[start:end] for start, end in relation["result"]]
        relations.append(
            Relation(
                type=type,
                causes=causes,
                effects=effects,
            )
        )

    processed.append(
        Example(
            id=id,
            text=text,
            relations=relations,
        )
    )

processed[:3]

[Example(id=2771, text="If one or more of Ecolab's customers were to experience a disastrous outcome, the firm's reputation could suffer and it could lose multiple customers as a result.", relations=[Relation(type='cause', causes=["one or more of Ecolab's customers were to experience a disastrous outcome"], effects=["the firm's reputation could suffer and it could lose multiple customers"])]),
 Example(id=3940, text="As grocery customers regularly visit the store, they are continually exposed to the firm's higher margin offerings, spurring lucrative general merchandise sales.", relations=[Relation(type='cause', causes=["hey are continually exposed to the firm's higher margin offerings"], effects=['spurring lucrative general merchandise sales'])]),
 Example(id=710, text='Nevertheless, with voices amplified through structural shifts like the rise of digital media, consumers have more agency than ever: if they want LaCroix (or any other National Beverage brand), retailers eventually have 

In [6]:
from IPython.display import display_markdown


def _render_example(ex: Example) -> str:
    out = [
        f"# ID: {ex.id}",
        ex.text,
        "",
        "# Relations",
        "",
    ]
    for i, rel in enumerate(ex.relations):
        out.extend(
            [
                f"## {i+1}: {rel.type}",
                "## Causes",
                "\n".join(f"{i+1}: {x}" for i, x in enumerate(rel.causes)),
                "",
                "## Effects",
                "\n".join(f"{i+1}: {x}" for i, x in enumerate(rel.effects)),
                "",
            ]
        )
    return "\n".join(out)


def render_example(ex: Example | list[Example]) -> str:
    if not isinstance(ex, list):
        ex = [ex]

    sep = "-" * 10
    return f"\n{sep}\n\n".join(_render_example(e) for e in ex)


def print_example(ex) -> None:
    print(render_example(ex))


# display_markdown(render_example(processed[0:20]), raw=True)
print_example(processed[0:20])

# ID: 2771
If one or more of Ecolab's customers were to experience a disastrous outcome, the firm's reputation could suffer and it could lose multiple customers as a result.

# Relations

## 1: cause
## Causes
1: one or more of Ecolab's customers were to experience a disastrous outcome

## Effects
1: the firm's reputation could suffer and it could lose multiple customers

----------

# ID: 3940
As grocery customers regularly visit the store, they are continually exposed to the firm's higher margin offerings, spurring lucrative general merchandise sales.

# Relations

## 1: cause
## Causes
1: hey are continually exposed to the firm's higher margin offerings

## Effects
1: spurring lucrative general merchandise sales

----------

# ID: 710
Nevertheless, with voices amplified through structural shifts like the rise of digital media, consumers have more agency than ever: if they want LaCroix (or any other National Beverage brand), retailers eventually have to oblige.

# Relations

## 1: ca